In [11]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from utils import *
import pandas as pd


# Carrega o modelo treinado

In [12]:
model = tf.keras.models.load_model('modelos/model_vgg_v2.keras')

model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,292,165 (62.15 MB)

 Trainable params: 525,825 (2.01 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 1,051,652 (4.01 MB)

# Definição das vagas

In [13]:
video = 'curto'
offset = 0 if video == 'curto' else 70

vagas = np.array([
    [(79, 394 + offset), (288, 667 + offset)],
    [(526, 394 + offset),(694, 667 + offset)],
    [(727, 394 + offset), (979, 667 + offset)],
    [(1001, 394 + offset), (1176, 667 + offset)],
    [(1215, 394 + offset), (1378, 667 + offset)],
    [(1422, 394 + offset), (1591, 667 + offset)]
], np.int32)

crop = [None for _ in vagas]

sift = cv2.SIFT_create()


# Aplicação do método em tempo real

In [10]:
video_path = f"videos/{video}.mp4"  

# Função de callback para capturar o clique do mouse
def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:  
        print(f"Coordenadas: ({x}, {y})")

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Erro ao abrir o vídeo")
else:
    cv2.namedWindow('Video')
    cv2.setMouseCallback('Video', mouse_callback)
    fps = cap.get(cv2.CAP_PROP_FPS)

    tempo_ocupado = [0 for _ in vagas]
    frames_ocupados = [0 for _ in vagas]
    ocupacao = [0 for _ in vagas]
    
    contador = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            novo_frame = frame.copy()

            if contador == 0:
                for i, vaga in enumerate(vagas):
                    p = calcular_pontos_retangulo(vaga[0], vaga[1])
                    crop[i] = cv2.cvtColor(cv2.resize(frame[p[0]:p[1], p[2]:p[3]], (224, 224)), cv2.COLOR_BGR2RGB)
                    
                    #cv2.imshow(f'crop_{i}', crop[i])

                pred = model.predict(np.array(crop))

            if contador < 30:
                contador += 1
            else:
                contador = 0
            
            for i, vaga in enumerate(vagas):
                if pred[i] >= 0.5:
                    frames_ocupados[i] += 1
                    ocupacao[i] = 1
                else:
                    ocupacao[i] = 0
                
                tempo_ocupado[i] = frames_ocupados[i] // fps

                cv2.rectangle(novo_frame, vaga[0], vaga[1], color=(0, 255, 0) if not ocupacao[i] else (0, 0, 255), thickness=2)

                novo_frame = cv2.putText(novo_frame, f'{int(tempo_ocupado[i] // 60):02}:{int(tempo_ocupado[i] % 60):02}', 
                                         ponto_central_retangulo(vaga[0], vaga[1]), cv2.FONT_HERSHEY_SIMPLEX, 
                                         1, (0, 255, 0) if not ocupacao[i] else (0, 0, 255), 2, cv2.LINE_AA)
            
            novo_frame = cv2.putText(novo_frame,f'Total de vagas: {len(vagas)}', (1566, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            novo_frame = cv2.putText(novo_frame,f'Vagas ocupadas: {np.sum(ocupacao)}', (1566, 90), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA) 
            novo_frame = cv2.putText(novo_frame,f'Ocupacao: {np.sum(ocupacao) / len(vagas) * 100:.2f}%', (1566, 130), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA) 

            cv2.imshow('Video', novo_frame)

            if cv2.waitKey(3) & 0xFF == ord('q'):
                break
        else:
            break

cap.release()
cv2.destroyAllWindows()

NameError: name 'video' is not defined

# Aplica o método e salva o vídeo resultante

In [19]:
video_path = f"videos/{video}.mp4" 
output_path = f"videos/resultado_NN_{video}.avi"

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Erro ao abrir o vídeo")
else:

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(fps)

    fourcc =  cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    tempo_ocupado = [0 for _ in vagas]
    frames_ocupados = [0 for _ in vagas]
    ocupacao = [0 for _ in vagas]
    indices = []
    serie_temporal = []

    contador = 0
    while cap.isOpened():
        ret, frame = cap.read()
        
        if ret:

            novo_frame = frame.copy()

            if contador == 0:
                for i, vaga in enumerate(vagas):
                    p = calcular_pontos_retangulo(vaga[0], vaga[1])
                    crop[i] = cv2.cvtColor(cv2.resize(frame[p[0]:p[1], p[2]:p[3]], (224, 224)), cv2.COLOR_BGR2RGB)
                    
                    #cv2.imshow(f'crop_{i}', crop[i])

                pred = model.predict(np.array(crop))

            if contador < 15:
                contador += 1
            else:
                contador = 0
            
            for i, vaga in enumerate(vagas):
                if pred[i] >= 0.5:
                    frames_ocupados[i] += 1
                    ocupacao[i] = 1
                else:
                    ocupacao[i] = 0
                
                tempo_ocupado[i] = frames_ocupados[i] // fps

                cv2.rectangle(novo_frame, vaga[0], vaga[1], color=(0, 255, 0) if not ocupacao[i] else (0, 0, 255), thickness=2)

                novo_frame = cv2.putText(novo_frame, f'{int(tempo_ocupado[i] // 60):02}:{int(tempo_ocupado[i] % 60):02}', 
                                         ponto_central_retangulo(vaga[0], vaga[1]), cv2.FONT_HERSHEY_SIMPLEX, 
                                         1, (0, 255, 0) if not ocupacao[i] else (0, 0, 255), 2, cv2.LINE_AA)
                
            novo_frame = cv2.putText(novo_frame,f'Total de vagas: {len(vagas)}', (1566, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            novo_frame = cv2.putText(novo_frame,f'Vagas ocupadas: {np.sum(ocupacao)}', (1566, 90), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA) 
            novo_frame = cv2.putText(novo_frame,f'Ocupacao: {np.sum(ocupacao) / len(vagas) * 100:.2f}%', (1566, 130), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            indices.append(cap.get(cv2.CAP_PROP_POS_FRAMES) / fps)
            serie_temporal.append(ocupacao.copy())

            out.write(novo_frame)

        else:
            print(cap.get(cv2.CAP_PROP_POS_FRAMES) / cap.get(cv2.CAP_PROP_FRAME_COUNT))
            break

cap.release()
out.release()

29.97002997002997
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 563ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 557ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 541ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 542ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 557ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 544ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 544ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 554ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 608ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 552ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 524ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 694ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 587ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 584ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 633ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 611ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 596ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 705ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 595ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 614ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 564ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 587ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 579ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 540ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 579ms/step
1/1 ━━━

# Exporta dados csv

In [148]:
pd.DataFrame(serie_temporal, index=indices).to_csv(f'dados/{video}_NN.csv')